Задание 1

Последовательная реализация на CPU

In [1]:
%%writefile consecutive.cpp

#include <iostream>
#include <vector>
#include <chrono>

int main() {
    // Размер массива
    const int N = 100000;

    // Инициализация массива значениями 1
    std::vector<int> a(N, 1);

    // Фиксация времени начала вычислений
    auto start = std::chrono::high_resolution_clock::now();

    // Последовательное суммирование элементов массива
    long long cpu_sum = 0;
    for (int i = 0; i < N; i++) {
        cpu_sum += a[i];
    }

    // Фиксация времени окончания вычислений
    auto end = std::chrono::high_resolution_clock::now();

    // Вычисление затраченного времени в миллисекундах
    std::chrono::duration<double, std::milli> cpu_time = end - start;

    // Вывод результата и времени выполнения
    std::cout << "CPU sum: " << cpu_sum << std::endl;
    std::cout << "CPU time: " << cpu_time.count() << " ms" << std::endl;

    return 0;
}

Writing consecutive.cpp


In [2]:
!g++ consecutive.cpp -o consecutive
!./consecutive

CPU sum: 100000
CPU time: 0.325889 ms


CUDA-реализация с использованием глобальной памяти

In [14]:
%%writefile cuda.cu

#include <iostream>
#include <cuda_runtime.h>

// CUDA-ядро для суммирования элементов массива
__global__ void sumKernel(int* d_a, int* d_sum, int n) {
    // Глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверка выхода за границы массива
    if (idx < n) {
        // Атомарное добавление элемента массива к общей сумме
        // Используется глобальная память
        atomicAdd(d_sum, d_a[idx]);
    }
}

int main() {
    // Размер массива
    const int N = 100000;
    const int size = N * sizeof(int);

    // Выделение и инициализация массива на host
    int* h_a = new int[N];
    for (int i = 0; i < N; i++) {
        h_a[i] = 1;
    }

    // Указатели на память device
    int* d_a;
    int* d_sum;

    // Выделение памяти на GPU
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_sum, sizeof(int));

    // Копирование данных с host на device
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);

    // Обнуление суммы на device
    cudaMemset(d_sum, 0, sizeof(int));

    // Конфигурация запуска ядра
    int threadsPerBlock = 256;
    int blocks = (N + threadsPerBlock - 1) / threadsPerBlock;

    // Создание CUDA-событий для замера времени
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Запуск таймера и CUDA-ядра
    cudaEventRecord(start);
    sumKernel<<<blocks, threadsPerBlock>>>(d_a, d_sum, N);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Получение времени выполнения ядра
    float gpu_time = 0.0f;
    cudaEventElapsedTime(&gpu_time, start, stop);

    // Копирование результата обратно на host
    int gpu_sum;
    cudaMemcpy(&gpu_sum, d_sum, sizeof(int), cudaMemcpyDeviceToHost);

    // Вывод результата и времени выполнения
    std::cout << "GPU sum: " << gpu_sum << std::endl;
    std::cout << "GPU time: " << gpu_time << " ms" << std::endl;

    // Освобождение памяти
    cudaFree(d_a);
    cudaFree(d_sum);
    delete[] h_a;

    return 0;
}

Overwriting cuda.cu


In [15]:
!nvcc cuda.cu -arch=compute_75 -code=sm_75 -o cuda
!./cuda

GPU sum: 100000
GPU time: 0.088096 ms


В обоих случаях получена одинаковая сумма элементов массива, равная 100000, что подтверждает корректность реализации как последовательного алгоритма на CPU, так и CUDA-версии на GPU.

Время выполнения на GPU оказалось меньше, чем на CPU. Это связано с тем, что вычисление суммы выполняется параллельно большим количеством потоков, тогда как CPU-реализация использует один поток. Несмотря на использование атомарных операций в глобальной памяти, которые снижают эффективность параллелизма, GPU показывает более высокую производительность для данной задачи.

Следует отметить, что измеряемое время на GPU включает только время выполнения CUDA-ядра и не учитывает накладные расходы на выделение памяти и копирование данных между host и device. При их учёте преимущество GPU для массива такого размера может уменьшиться.

Задание 2

Последовательная реализация на CPU

In [1]:
%%writefile consecutive_scan.cpp

#include <iostream>
#include <vector>
#include <chrono>

int main() {
    // Размер массива
    const int N = 1'000'000;

    // Входной массив и массив префиксных сумм
    std::vector<int> a(N, 1);
    std::vector<int> prefix(N);

    // Фиксация времени начала вычислений
    auto start = std::chrono::high_resolution_clock::now();

    // Последовательное вычисление префиксной суммы
    prefix[0] = a[0];
    for (int i = 1; i < N; i++) {
        prefix[i] = prefix[i - 1] + a[i];
    }

    // Фиксация времени окончания вычислений
    auto end = std::chrono::high_resolution_clock::now();

    // Вычисление времени выполнения в миллисекундах
    std::chrono::duration<double, std::milli> cpu_time = end - start;

    // Вывод последнего элемента (проверка корректности)
    std::cout << "CPU last element: " << prefix[N - 1] << std::endl;
    std::cout << "CPU time: " << cpu_time.count() << " ms" << std::endl;

    return 0;
}

Writing consecutive_scan.cpp


In [2]:
!g++ consecutive_scan.cpp -o consecutive_scan
!./consecutive_scan

CPU last element: 1000000
CPU time: 7.72496 ms


CUDA-реализация с shared memory

In [17]:
%%writefile cuda_scan.cu

#include <iostream>
#include <cuda_runtime.h>

/*
 * Kernel выполняет префиксную сумму внутри одного блока.
 * Для ускорения используется разделяемая память (shared memory).
 * Также сохраняется сумма элементов каждого блока.
 */
__global__ void scanKernel(int* d_in, int* d_out, int* block_sums, int n) {
    // Разделяемая память для текущего блока
    extern __shared__ int temp[];

    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + tid;

    // Загрузка данных из глобальной памяти в shared memory
    if (gid < n)
        temp[tid] = d_in[gid];
    else
        temp[tid] = 0;

    __syncthreads();

    // Параллельный scan (алгоритм Hillis–Steele)
    for (int offset = 1; offset < blockDim.x; offset <<= 1) {
        int value = 0;
        if (tid >= offset)
            value = temp[tid - offset];

        __syncthreads();
        temp[tid] += value;
        __syncthreads();
    }

    // Запись локального результата обратно в глобальную память
    if (gid < n)
        d_out[gid] = temp[tid];

    // Последний поток блока сохраняет сумму элементов блока
    if (tid == blockDim.x - 1)
        block_sums[blockIdx.x] = temp[tid];
}

/*
 * Kernel добавляет к каждому элементу сумму всех предыдущих блоков
 */
__global__ void addBlockSums(int* d_out, int* block_prefix, int n) {
    int gid = blockIdx.x * blockDim.x + threadIdx.x;

    if (blockIdx.x > 0 && gid < n) {
        d_out[gid] += block_prefix[blockIdx.x - 1];
    }
}

int main() {
    // Размер массива
    const int N = 1'000'000;
    const int size = N * sizeof(int);

    // Параметры запуска CUDA
    const int threadsPerBlock = 256;
    const int blocks = (N + threadsPerBlock - 1) / threadsPerBlock;

    // Выделение и инициализация входного массива на host
    int* h_in = new int[N];
    for (int i = 0; i < N; i++)
        h_in[i] = 1;

    // Указатели на память device
    int *d_in, *d_out, *d_block_sums;

    cudaMalloc(&d_in, size);
    cudaMalloc(&d_out, size);
    cudaMalloc(&d_block_sums, blocks * sizeof(int));

    // Копирование данных с host на device
    cudaMemcpy(d_in, h_in, size, cudaMemcpyHostToDevice);

    // CUDA-события для замера времени выполнения
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);

    // 1. Вычисление префиксной суммы внутри каждого блока
    scanKernel<<<blocks, threadsPerBlock, threadsPerBlock * sizeof(int)>>>(
        d_in, d_out, d_block_sums, N);

    // 2. Копирование сумм блоков на host
    int* h_block_sums = new int[blocks];
    cudaMemcpy(h_block_sums, d_block_sums,
               blocks * sizeof(int), cudaMemcpyDeviceToHost);

    // Последовательное вычисление префиксной суммы блоков
    for (int i = 1; i < blocks; i++)
        h_block_sums[i] += h_block_sums[i - 1];

    // Копирование результатов обратно на device
    cudaMemcpy(d_block_sums, h_block_sums,
               blocks * sizeof(int), cudaMemcpyHostToDevice);

    // 3. Добавление сумм предыдущих блоков
    addBlockSums<<<blocks, threadsPerBlock>>>(d_out, d_block_sums, N);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Получение времени выполнения GPU-версии
    float gpu_time = 0.0f;
    cudaEventElapsedTime(&gpu_time, start, stop);

    // Копирование последнего элемента для проверки корректности
    int last;
    cudaMemcpy(&last, &d_out[N - 1], sizeof(int),
               cudaMemcpyDeviceToHost);

    std::cout << "GPU last element: " << last << std::endl;
    std::cout << "GPU time: " << gpu_time << " ms" << std::endl;

    // Освобождение памяти
    cudaFree(d_in);
    cudaFree(d_out);
    cudaFree(d_block_sums);
    delete[] h_in;
    delete[] h_block_sums;

    return 0;
}


Writing cuda_scan.cu


In [18]:
!nvcc cuda_scan.cu -arch=compute_75 -code=sm_75 -o cuda_scan
!./cuda_scan

GPU last element: 1000000
GPU time: 0.33136 ms


Обе реализации корректно вычисляют префиксную сумму, что подтверждается совпадением результатов. Последовательная реализация на CPU выполнилась за 7.72 мс, тогда как CUDA-реализация с использованием разделяемой памяти показала время 0.33 мс, обеспечив ускорение более чем в 20 раз. Полученный выигрыш объясняется эффективным распараллеливанием вычислений и снижением числа обращений к глобальной памяти за счёт использования shared memory.

Задание 3

Последовательная реализация на CPU

In [10]:
%%writefile sequential3.cpp
#include <iostream>
#include <vector>
#include <chrono>

// Функция последовательного суммирования элементов массива на CPU
long long cpu_sum(const std::vector<int>& a) {
    long long sum = 0;
    for (int x : a)
        sum += x;
    return sum;
}

int main() {
    // Размер массива
    const int N = 1'000'000;

    // Инициализация массива значениями 1
    std::vector<int> a(N, 1);

    // Фиксация времени начала вычислений
    auto start = std::chrono::high_resolution_clock::now();

    // Последовательное вычисление суммы на CPU
    long long result = cpu_sum(a);

    // Фиксация времени окончания вычислений
    auto end = std::chrono::high_resolution_clock::now();

    // Вычисление времени выполнения
    std::chrono::duration<double, std::milli> time = end - start;

    // Вывод результата и времени
    std::cout << "CPU sum: " << result << std::endl;
    std::cout << "CPU time: " << time.count() << " ms" << std::endl;

    return 0;
}

Writing sequential3.cpp


In [11]:
!g++ sequential3.cpp -o sequential3
!./sequential3

CPU sum: 1000000
CPU time: 17.2313 ms


Реализация на GPU (глобальная память + atomicAdd)

In [3]:
%%writefile gpu3.cu

#include <iostream>
#include <cuda_runtime.h>

// CUDA-ядро для суммирования элементов массива
// Каждый поток обрабатывает один элемент
__global__ void sumKernel(int* d_a, int* d_sum, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверка выхода за границы массива
    if (idx < n) {
        // Атомарное добавление значения к общей сумме
        atomicAdd(d_sum, d_a[idx]);
    }
}

int main() {
    // Размер массива
    const int N = 1'000'000;
    const int size = N * sizeof(int);

    // Выделение и инициализация массива на host
    int* h_a = new int[N];
    for (int i = 0; i < N; i++)
        h_a[i] = 1;

    // Указатели на память device
    int *d_a, *d_sum;

    // Выделение памяти на GPU
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_sum, sizeof(int));

    // Копирование данных с host на device
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);

    // Инициализация суммы нулём
    cudaMemset(d_sum, 0, sizeof(int));

    // Параметры запуска CUDA-ядра
    int threads = 256;
    int blocks = (N + threads - 1) / threads;

    // CUDA-события для замера времени выполнения
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Запуск таймера и CUDA-ядра
    cudaEventRecord(start);
    sumKernel<<<blocks, threads>>>(d_a, d_sum, N);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Получение времени выполнения ядра
    float gpu_time;
    cudaEventElapsedTime(&gpu_time, start, stop);

    // Копирование результата обратно на host
    int result;
    cudaMemcpy(&result, d_sum, sizeof(int), cudaMemcpyDeviceToHost);

    // Вывод результата и времени
    std::cout << "GPU sum: " << result << std::endl;
    std::cout << "GPU time: " << gpu_time << " ms" << std::endl;

    // Освобождение памяти
    cudaFree(d_a);
    cudaFree(d_sum);
    delete[] h_a;

    return 0;
}


Writing gpu3.cu


In [4]:
!nvcc gpu3.cu -arch=compute_75 -code=sm_75 -o gpu3
!./gpu3

GPU sum: 1000000
GPU time: 0.155456 ms


Гибридная реализация CPU + GPU

In [5]:
%%writefile hybrid.cu

#include <iostream>
#include <vector>
#include <chrono>
#include <cuda_runtime.h>

// CUDA-ядро для суммирования части массива
__global__ void sumKernel(int* d_a, int* d_sum, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        atomicAdd(d_sum, d_a[idx]);
    }
}

int main() {
    // Размер массива
    const int N = 1'000'000;

    // Половина массива для гибридной обработки
    const int half = N / 2;

    // Инициализация массива на host
    std::vector<int> a(N, 1);

    // Фиксация времени начала гибридных вычислений
    auto start = std::chrono::high_resolution_clock::now();

    // --- CPU: обработка первой половины массива ---
    long long cpu_part = 0;
    for (int i = 0; i < half; i++)
        cpu_part += a[i];

    // --- GPU: обработка второй половины массива ---
    int size = half * sizeof(int);
    int *d_a, *d_sum;

    // Выделение памяти на GPU
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_sum, sizeof(int));

    // Копирование второй половины массива на device
    cudaMemcpy(d_a, a.data() + half, size, cudaMemcpyHostToDevice);

    // Инициализация суммы нулём
    cudaMemset(d_sum, 0, sizeof(int));

    // Параметры запуска CUDA-ядра
    int threads = 256;
    int blocks = (half + threads - 1) / threads;

    // Запуск CUDA-ядра для второй половины массива
    sumKernel<<<blocks, threads>>>(d_a, d_sum, half);
    cudaDeviceSynchronize();

    // Копирование результата GPU-части на host
    int gpu_part;
    cudaMemcpy(&gpu_part, d_sum, sizeof(int), cudaMemcpyDeviceToHost);

    // Фиксация времени окончания вычислений
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> time = end - start;

    // Объединение результатов CPU и GPU
    long long total = cpu_part + gpu_part;

    // Вывод результата и времени
    std::cout << "Hybrid sum: " << total << std::endl;
    std::cout << "Hybrid time: " << time.count() << " ms" << std::endl;

    // Освобождение памяти
    cudaFree(d_a);
    cudaFree(d_sum);

    return 0;
}


Writing hybrid.cu


In [6]:
!nvcc hybrid.cu -arch=compute_75 -code=sm_75 -o hybrid
!./hybrid

Hybrid sum: 1000000
Hybrid time: 206.595 ms


Все три реализации корректно вычисляют сумму элементов массива, что подтверждается совпадением результатов. Последовательная реализация на CPU показала время выполнения 17.23 мс, тогда как реализация на GPU выполнилась значительно быстрее — за 0.16 мс благодаря массовому параллелизму.

Гибридная реализация оказалась существенно медленнее и выполнилась за 206.6 мс. Это объясняется большими накладными расходами на передачу данных между CPU и GPU, а также последовательным характером выполнения частей алгоритма. В данной задаче вычислительная нагрузка слишком мала, чтобы компенсировать стоимость копирования данных и синхронизации.

Полученные результаты показывают, что гибридный подход не всегда приводит к ускорению и эффективен только при достаточно сложных вычислениях или при перекрытии вычислений и обмена данными.

Задание 4

In [1]:
%%writefile mpi_sum.cpp

#include <mpi.h>
#include <iostream>
#include <vector>

int main(int argc, char** argv) {
    // Инициализация MPI-среды
    MPI_Init(&argc, &argv);

    int rank, size;
    // Получение номера текущего процесса
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    // Получение общего числа процессов
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Размер обрабатываемого массива
    const int N = 1'000'000;

    // Размер части массива для каждого процесса
    int chunk_size = N / size;

    // Основной массив (используется только процессом с rank 0)
    std::vector<int> data;

    // Локальная часть массива для каждого процесса
    std::vector<int> local_data(chunk_size);

    // Инициализация массива только на главном процессе
    if (rank == 0) {
        data.resize(N, 1);
    }

    // Фиксация времени начала выполнения
    double start_time = MPI_Wtime();

    // Распределение массива между процессами
    // Каждый процесс получает chunk_size элементов
    MPI_Scatter(
        data.data(),            // исходный массив (только у rank 0)
        chunk_size, MPI_INT,    // количество и тип отправляемых данных
        local_data.data(),      // локальный массив
        chunk_size, MPI_INT,    // количество и тип принимаемых данных
        0,                      // root-процесс
        MPI_COMM_WORLD
    );

    // Локальное вычисление суммы элементов
    long long local_sum = 0;
    for (int value : local_data) {
        local_sum += value;
    }

    // Переменная для хранения итоговой суммы
    long long global_sum = 0;

    // Сбор локальных сумм со всех процессов
    MPI_Reduce(
        &local_sum,             // локальное значение
        &global_sum,            // итоговое значение (у rank 0)
        1,
        MPI_LONG_LONG,
        MPI_SUM,
        0,
        MPI_COMM_WORLD
    );

    // Фиксация времени окончания выполнения
    double end_time = MPI_Wtime();

    // Вывод результатов только главным процессом
    if (rank == 0) {
        std::cout << "MPI processes: " << size << std::endl;
        std::cout << "Global sum: " << global_sum << std::endl;
        std::cout << "Execution time: "
                  << (end_time - start_time) * 1000
                  << " ms" << std::endl;
    }

    // Завершение MPI-среды
    MPI_Finalize();
    return 0;
}



Writing mpi_sum.cpp


In [2]:
!mpic++ mpi_sum.cpp -o mpi_sum

In [8]:
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_sum
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_sum
!mpirun --allow-run-as-root --oversubscribe -np 8 ./mpi_sum

MPI processes: 2
Global sum: 1000000
Execution time: 11.1409 ms
MPI processes: 4
Global sum: 1000000
Execution time: 19.5684 ms
MPI processes: 8
Global sum: 1000000
Execution time: 12.963 ms


Распределённая MPI-программа корректно вычисляет сумму элементов массива, что подтверждается совпадением результата при использовании 2, 4 и 8 процессов. При этом время выполнения не уменьшается с ростом числа процессов: при 2 процессах время составило 11.14 мс, при 4 — увеличилось до 19.57 мс, а при 8 — составило 12.96 мс.

Такое поведение объясняется тем, что вычисления выполнялись на одном узле с ограниченным числом CPU-ядер, а запуск большего числа MPI-процессов происходил в режиме oversubscribe. В результате накладные расходы на создание процессов, обмен данными и синхронизацию превысили выигрыш от параллелизма.

Полученные результаты показывают, что эффективность MPI-параллелизма напрямую зависит от аппаратных ресурсов и что распределённые вычисления оправданы в первую очередь для крупных задач и многопроцессорных систем, а не для небольших массивов на одной машине.